In [3]:
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import os

In [5]:
os.getcwd()

'D:\\machine_learning\\game\\kaggle\\kaggle-past-solution-reproduce'

In [8]:
#load raw data
#why drop 2016
df_train = pd.read_csv(
    'time_series/Grocery_Sales_Forecasting/data/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "time_series/Grocery_Sales_Forecasting/data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    "time_series/Grocery_Sales_Forecasting/data/items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "time_series/Grocery_Sales_Forecasting/data/stores.csv",
).set_index("store_nbr")

In [11]:
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [12]:
le = LabelEncoder()
items['family'] = le.fit_transform(items['family'].values)

stores['city'] = le.fit_transform(stores['city'].values)
stores['state'] = le.fit_transform(stores['state'].values)
stores['type'] = le.fit_transform(stores['type'].values)

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59038132 entries, 0 to 59038131
Data columns (total 5 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
unit_sales     float64
onpromotion    bool
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 1.8 GB


In [20]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [30]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

In [38]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [44]:
df_2017.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>date</th>
      <th>2017-01-01 00:00:00</th>
      <th>2017-01-02 00:00:00</th>
      <th>2017-01-03 00:00:00</th>
      <th>2017-01-04 00:00:00</th>
      <th>2017-01-05 00:00:00</th>
      <th>2017-01-06 00:00:00</th>
      <th>2017-01-07 00:00:00</th>
      <th>2017-01-08 00:00:00</th>
      <th>2017-01-09 00:00:00</th>
      <th>2017-01-10 00:00:00</th>
      <th>...</th>
      <th>2017-08-06 00:00:00</th>
      <th>2017-08-07 00:00:00</th>
      <th>2017-08-08 00:00:00</th>
      <th>2017-08-09 00:00:00</th>
      <th>2017-08-10 00:00:00</th>
      <th>2017-08-11 00:00:00</th>
      <th>2017-08-12 00:00:00</th>
      <th>2017-08-13 00:00:00</th>
      <th>2017-08-14 00:00:00</th>
      <th>2017-08-15 00:00:00</th>
    </tr>
    <tr>
      <th>store_nbr</th>
      <th>item_nbr</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">1</th>
      <th>96995</th>
      <td>0.0</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>...</td>
      <td>1.098612</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.693147</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>99197</th>
      <td>0.0</td>
      <td>0.000000</td>
      <td>1.386294</td>
      <td>0.693147</td>
      <td>0.693147</td>
      <td>0.693147</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.693147</td>
      <td>...</td>
      <td>0.000000</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>103520</th>
      <td>0.0</td>
      <td>0.693147</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>1.098612</td>
      <td>1.386294</td>
      <td>0.693147</td>
      <td>0.000000</td>
      <td>0.693147</td>
      <td>0.693147</td>
      <td>...</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>1.386294</td>
      <td>0.000000</td>
      <td>1.386294</td>
      <td>0.693147</td>
      <td>0.693147</td>
      <td>0.693147</td>
      <td>0.000000</td>
      <td>0.000000</td>
    </tr>
    <tr>
      <th>103665</th>
      <td>0.0</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>1.386294</td>
      <td>1.098612</td>
      <td>1.098612</td>
      <td>0.693147</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>2.079442</td>
      <td>...</td>
      <td>0.693147</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>2.079442</td>
      <td>2.302585</td>
      <td>1.098612</td>
      <td>0.000000</td>
      <td>0.000000</td>
      <td>0.693147</td>
      <td>0.693147</td>
    </tr>
    <tr>
      <th>105574</th>
      <td>0.0</td>
      <td>0.000000</td>
      <td>1.791759</td>
      <td>2.564949</td>
      <td>2.302585</td>
      <td>1.945910</td>
      <td>1.609438</td>
      <td>1.098612</td>
      <td>1.386294</td>
      <td>2.302585</td>
      <td>...</td>
      <td>0.000000</td>
   

In [44]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [45]:
items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))

In [50]:
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

In [58]:
df_2017_store_class = df_2017.reset_index()
df_2017_store_class['class'] = items['class'].values
df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

df_2017_promo_store_class = promo_2017.reset_index()
df_2017_promo_store_class['class'] = items['class'].values
df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

In [61]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]